In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from prettytable import PrettyTable 

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier as DT
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.neural_network import MLPClassifier
import xgboost as xgb

dataset = pd.read_csv('../input/heart-disease-cleveland-uci/heart_cleveland_upload.csv')
X = dataset.iloc[:,:-1].values
y = dataset.iloc[:,-1].values

#Encoding Categorical Data
#from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

#cp
labelencoder_X = LabelEncoder()

X[:,2] = labelencoder_X.fit_transform(X[:,2])
ct = ColumnTransformer([("cp", OneHotEncoder(), [2])],    remainder = 'passthrough') 
X = ct.fit_transform(X)

#restecg
#labelencoder_X = LabelEncoder()
#X[:,9] = labelencoder_X.fit_transform(X[:,9])
ct = ColumnTransformer([("restecg", OneHotEncoder(), [9])],    remainder = 'passthrough')
X = ct.fit_transform(X)

#slope
#labelencoder_X = LabelEncoder()
#X[:,15] = labelencoder_X.fit_transform(X[:,15])
ct = ColumnTransformer([("slope", OneHotEncoder(), [15])],    remainder = 'passthrough')
X = ct.fit_transform(X)

#ca
#labelencoder_X = LabelEncoder()
#X[:,18] = labelencoder_X.fit_transform(X[:,18])
ct = ColumnTransformer([("ca", OneHotEncoder(), [18])],    remainder = 'passthrough')
X = ct.fit_transform(X)

#thal
#labelencoder_X = LabelEncoder()
#X[:,23] = labelencoder_X.fit_transform(X[:,23])
ct = ColumnTransformer([("thal", OneHotEncoder(), [22])],    remainder = 'passthrough')
X = ct.fit_transform(X)

from sklearn.preprocessing import StandardScaler
scalerX = StandardScaler()
X = scalerX.fit_transform(X)

from sklearn.model_selection import train_test_split
XTrain, XTest, yTrain, yTest = train_test_split(X, y, test_size=0.3, random_state=0)

def ANN() :
    from keras.models import Sequential
    from keras.layers import Dense
    #Initialising ANN
    model = Sequential()
    #Adding the first hidden layer or the input layer
#25,12,24,1
#63,12,24,1
    model.add(Dense(activation='relu',
                         kernel_initializer='uniform',
                         input_dim=25,
                         units=12))
    #Adding the second hidden layer
    model.add(Dense(activation='relu',
                         kernel_initializer='uniform',
                         units=24))
    #classifier.add(Dense(activation='relu',
    #                    kernel_initializer='uniform',
    #                     units=50))

    model.add(Dense(activation='sigmoid',
                         kernel_initializer='uniform',
                         units=1))
    #Compiling the ANN
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    print(model.summary())
        history = model.fit(XTrain, yTrain, batch_size=5, epochs=35, verbose=0)
    #history = model.fit(X, y, batch_size=5, epochs=50, verbose=0)

    yPred = model.predict(XTest)
    yPred = (yPred>0.5) #Since output is probability
    mse = mean_squared_error(yTest,yPred)
    r = r2_score(yTest,yPred)
    mae = mean_absolute_error(yTest,yPred)
    accuracy = accuracy_score(yTest,yPred)
    print("Artificial Neural Network Classifier :")
    print("Accuracy = ", accuracy)
    print("Mean Squared Error:",mse)
    print("R score:",r)
    print("Mean Absolute Error:",mae)
    table.add_row( ['Artificial NN', format(accuracy, '.3f'),format(mse, '.3f'),format(r, '.3f'),format(mae, '.3f')])
   
table = PrettyTable()
table.field_names = ["Model","Accuracy", "Mean Squared Error", "R² score","Mean Absolute Error"]     

models = []
models.append(('LR', LogisticRegression()))
models.append(('KNN', KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski')))
models.append(('SVM', SVC(kernel='linear',random_state=0)))
models.append(('NB', GaussianNB()))
models.append(('DTC', DT(criterion='entropy', random_state=0)))
models.append(('RFC', RF(n_estimators=10, criterion='entropy', random_state=0)))
models.append(('PER', Perceptron(tol=1e-3, random_state=0)))
models.append(('MLPER', MLPClassifier(activation='logistic', max_iter=200, hidden_layer_sizes=(20,), alpha=0.001, solver='lbfgs', verbose = True)))
models.append(('XGB', xgb.XGBClassifier(max_depth=5, learning_rate=0.1,n_jobs=-1,use_label_encoder=False)))


for name, model in models:
    model.fit(XTrain, yTrain)
    #model.fit(X, y) 
    yPred = model.predict(XTest)
    accuracy = accuracy_score(yTest,yPred)
    mse = mean_squared_error(yTest,yPred)
    r = r2_score(yTest,yPred)
    mae = mean_absolute_error(yTest,yPred)
    table.add_row([type(model).__name__, format(accuracy, '.3f'),format(mse, '.3f'),format(r, '.3f'),format(mae, '.3f')])
  
ANN()
print(table)

